In [ ]:
'''
# IoU_old
usage of loss func

criterion = nn.NLLLoss()

def step(x, y, is_train=True):
  x = x.reshape(-1, 28 * 28)
  y_pred = model(x)
  loss = criterion(y_pred, y)
  if is_train:
    opt.zero_grad()
    loss.backward()
    opt.step()
  return loss, y_pred

'''
torch.nn.Module
class IoU_old(nn.Module):
  def __init__(self, thresh: float = 0.5):
    super().__init__()
    self.thresh = thresh

  def forward(self, inputs: torch.Tensor, targets:torch.Tensor, weights: Optional[torch.Tensor] = None, smooth: float = 0.0) -> Tensor:
    inputs = torch.where(inputs < self.thresh, 0, 1)
    batch_size = targets.shape[0]

    intersect = torch.logical_and(inputs, targets)
    intersect = intersect.view(batch_size, -1).sum(-1)
    union = torch.logical_or(inputs, targets)
    union = union.view(batch_size, -1).sum(-1)

    print(intersect)
    print(union)
    print(union.shape)

    IoU = (intersect + smooth) / (union + smooth)
    IoU = torch.nan_to_num(IoU)
    return IoU

In [ ]:
def iou2(outputs: torch.Tensor, labels: torch.Tensor, smooth: float=0, threshold: float = 0.5):

  # intersection = (outputs & labels).sum((1, 2))  # Will be zero if Truth=0 or Prediction=0
  # union = (outputs | labels).sum((1, 2))         # Will be zzero if both are 0
  # iou = (intersection + SMOOTH) / (union + SMOOTH)  # We smooth our devision to avoid 0/0
  # # thresholded = torch.clamp(20 * (iou - 0.5), 0, 10).ceil() / 10  # This is equal to comparing with thresolds

  # return iou
  outputs = torch.where(outputs < threshold, 0, 1)

  print(f"num of labels: {len(torch.nonzero(labels))}")
  intersect = torch.logical_and(outputs, labels)
  intersect = intersect.long()
  print(intersect)
  # intersect = torch.where(outputs == labels, 1, 0)
  # intersect = torch.where(outputs == intersect, 1, 0)
  print(f"num of interesect: {len(torch.nonzero(intersect))}")
  union = torch.logical_or(outputs, labels)

  print(f"num of union: {len(torch.nonzero(union))}")

  # print(f"intersect sum: {intersect.sum()}")
  # print(f"union sum: {union.sum()}")
  iou_score = torch.sum(intersect)/torch.sum(union)
  print(iou_score)
  #   # You can comment out this line if you are passing tensors of equal shape
  #   # But if you are passing output from UNet or something it will most probably
  #   # be with the BATCH x 1 x H x W shape
  #   # outputs = outputs.squeeze(1)  # BATCH x 1 x H x W => BATCH x H x W
    
  #   # intersection = (outputs & labels).sum((1, 2))  # Will be zero if Truth=0 or Prediction=0
  #   # union = (outputs | labels).float().sum((1, 2))         # Will be zzero if both are 0
    
  iou = (intersect + smooth) / (union + smooth)  # We smooth our devision to avoid 0/0
  iou_score = torch.sum(iou)
  print(iou_score)
      
  return iou_score  # Or thresholded.mean() if you are interested in average across the batch

In [ ]:
'''
# IoU soft
note: "hard" probabilistic IoU calculation is not differentiable, 
so be sure to use soft probabilistic loss func
'''
def iou_soft(inputs: torch.Tensor, labels: torch.Tensor, 
             smooth: float=0.1, threshold: float = 0.5, alpha: float = 1.0):
  
  '''
  - alpha: a parameter that sharpen the thresholding.
    if alpha = 1 -> thresholded input is the same as raw input.
  '''

  thresholded_inputs = inputs**alpha / (inputs**alpha + (1 - inputs)**alpha)
  # inputs = torch.where(thresholded_inputs < threshold, 0, 1)

  thresholded_inputs = torch.where(thresholded_inputs < threshold, 0, 1)
  inputs = (inputs + thresholded_inputs) - inputs.detach()

  batch_size = inputs.shape[0]

  # instead of hard prob calc: intersect = torch.logical_and(outputs, labels)
  intersect_tensor = (inputs * labels).view(batch_size, -1)
  intersect = intersect_tensor.sum(-1)

  # insetad of using the hard prob: union = torch.logical_or(outputs, labels)   
  union_tensor = torch.max(inputs, labels).view(batch_size, -1)
  union = union_tensor.sum(-1)


  iou = (intersect + smooth) / (union + smooth)  # We smooth our devision to avoid 0/0
  iou_score = iou.mean()

  # print(f"num of labels: {len(torch.nonzero(labels))}")
  # print(f"intersection tensor: {intersect_tensor}")
  # print(f"union tensor: {union_tensor}\n")
  # print(f"intersect sum: {intersect}")
  # print(f"union sum: {union}\n")
  # print(f"iou error: {1- iou_score}")
      
  return 1- iou_score
